In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager

print(plt.style.available)
%matplotlib inline
driver = webdriver.Chrome()

url='https://cq.zu.fang.com/house-a060-b04863/'
driver.get(url)

# 初始化列表存储数据
data = []

# 抓取20页数据
for page_num in range(1, 21):
    print(f"正在抓取第 {page_num} 页数据...")
    
    # 等待页面加载
    sleep(3)
    
    try:
        table = driver.find_element('class name','list-box')
        rows = table.find_elements(By.TAG_NAME, 'dl')
        
        print(f"本页找到 {len(rows)} 个房源")
        
        # 重要：遍历当前页的每个房源
        for i, row in enumerate(rows):
            # 调试：查看第一个房源的完整信息
            if i == 0 and page_num == 1:
                print("=== 第一个房源的完整文本 ===")
                print(row.text)
                print("="*50)
            
            try:
                # 提取价格 - 使用相对XPath
                price = row.find_element(By.XPATH, './/span[contains(@class, "price")]').text
            except NoSuchElementException:
                price = "价格未知"
            
            # 修改的面积提取逻辑 - 更简单直接的方法
            area = "面积未知"
            try:
                # 先尝试方法2：从房屋信息p标签中提取
                info_element = row.find_element(By.XPATH, './/p[contains(@class, "font15")]')
                info_text = info_element.text
                
                # 在完整信息文本中查找"㎡"并提取前面的数字
                if '㎡' in info_text:
                    # 找到"㎡"的位置，然后向前找数字
                    import re
                    area_match = re.search(r'(\d+)\s*㎡', info_text)
                    if area_match:
                        area = area_match.group(1) + '㎡'
                    else:
                        # 如果正则匹配失败，使用分割方法
                        area_parts = info_text.split('㎡')
                        if len(area_parts) > 1:
                            # 取"㎡"前面的部分，然后取最后一个单词（应该是数字）
                            before_m2 = area_parts[0].strip()
                            area_num = before_m2.split()[-1] if before_m2.split() else "未知"
                            area = area_num + '㎡'
                
            except NoSuchElementException:
                # 如果方法2失败，尝试方法1：直接找包含"㎡"的元素
                try:
                    area_element = row.find_element(By.XPATH, './/*[contains(text(), "㎡")]')
                    area_text = area_element.text
                    if '㎡' in area_text:
                        area_parts = area_text.split('㎡')
                        if len(area_parts) > 1:
                            area_num = area_parts[0].strip()
                            area = area_num + '㎡'
                except NoSuchElementException:
                    area = "面积未知"

            row_data = [price, area]
            data.append(row_data)
            print(f"  房源 {i+1}: 价格={price}, 面积={area}")
            
    except Exception as e:
        print(f"第 {page_num} 页数据提取失败: {e}")
        continue
    
    # 如果不是最后一页，则点击下一页
    if page_num < 20:
        try:
            next_page = driver.find_element(By.XPATH, '//*[@id="rentid_D10_01"]/a[7]')
            next_page.click()
            print("成功点击下一页")
        except NoSuchElementException:
            print("没有找到下一页按钮，提前结束抓取")
            break
    else:
        print("已达到20页，停止抓取")

# 创建DataFrame
df = pd.DataFrame(data, columns=['价格', '面积'])
print(f"总共抓取了 {len(df)} 条数据")
print(df.head())

# 保存数据
df.to_csv('租房数据.csv', index=False, encoding='utf-8-sig')
print("数据已保存到 租房数据.csv")

driver.quit()

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']
正在抓取第 1 页数据...
本页找到 60 个房源
=== 第一个房源的完整文本 ===
沙坪坝区 富力城天禧 三房
整租|3室2厅|98㎡|朝南
沙坪坝-大学城-富力城天禧
1600元/月
  房源 1: 价格=1600, 面积=98㎡
  房源 2: 价格=1900, 面积=73㎡
  房源 3: 价格=1900, 面积=87㎡
  房源 4: 价格=1800, 面积=114㎡
  房源 5: 价格=1700, 面积=125㎡
  房源 6: 价格=1800, 面积=88㎡
  房源 7: 价格=1600, 面积=76㎡
  房源 8: 价格=1700, 面积=94㎡
  房源 9: 价格=1400, 面积=73㎡
  房源 10: 价格=1100, 面积=68㎡
  房源 11: 价格=1400, 面积=85㎡
  房源 12: 价格=2000, 面积=96㎡
  房源 13: 价格=1800, 面积=113㎡
  房源 14: 价格=1200, 面积=40㎡
  房源 15: 价格=1700, 面积=113㎡
  房源 16: 价格=1300, 面积=69㎡
  房源 17: 价格=1100, 面积=35㎡
  房源 18: 价格=1100, 面积=39㎡
  房源 19: 价格=1000